In [201]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold
from sklearn.feature_selection import RFECV, f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from functools import reduce

In [68]:
def impute(X, method):
    """
    impute missing values with mean or median
    
    parameters
    -----------
    X : Pandas Dataframe
    method: String for imputation method. supports 'mean' and 'median'.
    """
    if method == 'mean':
        return X.apply(lambda x: x.fillna(x.mean()))
    elif method == 'median':
        return X.apply(lambda x: x.fillna(x.median()))
    else:
        raise 'unsupported method'

In [165]:
def cross_validate_and_impute(model, X, y, n_folds, scoring_metric, impute_method, random_state):
    """
    evaluates the model by cross validation, imputes missing values within cross-validation iterations
    """
    estimator = Pipeline([('imputer', Imputer(strategy=impute_method)), ('model', model)])
    cv = StratifiedKFold(n_splits = n_folds, shuffle=False, random_state = random_state)
    scores = cross_val_score(estimator, X, y, scoring = scoring_metric, cv = cv)
    return scores

In [46]:
def determine_attribute_sets(d_names):
    """
    Determine unique attribute types and expanded sets
    """
    attribute_types = []  # unique attribute root name
    for name in d_names:
        if name.split('_')[0] not in attribute_types:
            attribute_types.append(name.split('_')[0])

    d_sets_of_attributes = []  # expanded set of an attribute
    d_names_for_attribute_sets = []  # names for each attribute in an expanded set
    for attribute_type in attribute_types:
        curr_attribute_columns = []
        curr_attribute_names = []
        for idx, name in enumerate(d_names):
            if name.split('_')[0] == attribute_type:
                curr_attribute_columns.append(idx)
                curr_attribute_names.append(name)

        d_sets_of_attributes.append(curr_attribute_columns)
        d_names_for_attribute_sets.append(curr_attribute_names)

    return [d_sets_of_attributes, d_names_for_attribute_sets]


In [161]:
def staged_feature_inclusion(X, y, sets_of_attributes, models_to_use, random_state):
    """
    This function runs staged_feature_inclusion method of feature selection. 
    It is intended for use when muliple features are constructed from a single variable.
    E.g., a time searies of heart rate measurements espanded into min, max, slopes, etc.
    The base variable must start the variable name and be underscore separated 
    (e.g., heatrate_max_value and heartrate_most_recent_value).

    sets_of_attributes is the first returned item from determine_attribute_sets()
    """
#     print(out_file)
#     out_f = open(out_file, 'w')
    models = {}
    models['lr'] = LogisticRegression(penalty='l2', random_state= random_state)
#     models['sv'] = SVC(C=1, probability=True, random_state=random_state)
#     models['rf'] = clf_rf = RandomForestClassifier(random_state=random_state)
    for model_name in models_to_use:
        informative_attributes = []
        rfecv = RFECV(estimator=models[model_name], step=1, scoring='roc_auc')
        # determine keep columns
        for idx_of_attributes in sets_of_attributes:
            x_current = X.iloc[:, idx_of_attributes]
#             try:
            # ## determine staged inclusion for even rows
            #scores = cross_validate(models[model_name], x_current, y, cv=3, scoring='roc_auc', return_train_score=False)
            scores = cross_validate_and_impute(models[model_name], x_current, y, n_folds=3,
                                               scoring_metric='roc_auc', impute_method = 'mean', random_state= random_state)
            if scores.mean() > 0.55:  # determine if set should be kept
                rfecv.fit(x_current, y)  # recursive feature elimination
                if rfecv.grid_scores_.mean() > 0.6:
                    informative_attributes += list(np.array(idx_of_attributes)[rfecv.support_])
                    # ^keep most important features
#             except (ValueError, IndexError):
#                 pass
    return informative_attributes


In [158]:
out_file = '..\output\selected_features.txt'

In [228]:
features = pd.read_csv("../data/all_features_178_patients.csv", index_col=0)

In [230]:
features.shape

(178, 19807)

In [229]:
targets = pd.read_csv("../data/targets.csv")

In [231]:
target_grps = [['K', 'CL', 'CO2', 'NA'],
               ['CREAT', 'BUN'],
               ['INR', 'PT']]

In [232]:
labels = []
for grp in target_grps:
    labels += [l for l in grp] 

In [233]:
selected_features = {}
for label in labels:
    
    ## select rows where target is not null
    mask = targets[label].notna().values
    y = targets.loc[mask, label]
    X = features.loc[mask, :].copy()
  
    ## drop all-null columns
    X.dropna(axis=1, how='all', inplace=True)
    
    ## impute mising    
    X = impute(X, method='mean')
    
    print('target=', label)
    print('features dimension=', X.shape)
    
    ## ANOVA feature-selection
    f, pval = f_classif(X, y)
    f = pd.Series(f).replace([np.inf, -np.inf], np.nan)
    pval[f.isna()] = np.nan
    
    ## select features with p-val<0.05
    selected_features[label] = X.columns[pval < 0.05]
    
    print('#selected features=', selected_features[label].size)
    print('#'*40)

target= K
features dimension= (177, 19756)
#selected features= 731
########################################


C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [   11    18    21 ... 19706 19709 19746] are constant.
  UserWarning)
C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: RuntimeWarning: invalid value encountered in less


target= CL
features dimension= (177, 19756)
#selected features= 777
########################################


C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [   11    18    21 ... 19693 19706 19709] are constant.
  UserWarning)


target= CO2
features dimension= (177, 19756)
#selected features= 796
########################################


...

C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [   11    18    21 ... 19667 19670 19689] are constant.
  UserWarning)


target= INR
features dimension= (125, 18828)
#selected features= 735
########################################


C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [   11    18    21 ... 18767 18780 18783] are constant.
  UserWarning)


target= PT
features dimension= (125, 18828)
#selected features= 948
########################################


C:\Users\mot16\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [   11    18    21 ... 18767 18780 18783] are constant.
  UserWarning)


In [243]:
union_selected_features = reduce(np.union1d, tuple([features.tolist() for _, features in selected_features.items()]))

In [250]:
union_selected_features.size

3476

In [249]:
union_selected_features[0:15]

array(['1/2 NS + KCL 20 mEq_med-ever_occurred',
       '12/27-NH contacted to fax med list and l_med-days_since_first_value',
       '12/27-NH contacted to fax med list and l_med-days_since_last_value',
       '2/8/2011-MAR not sent from NH for last d_med-days_since_first_value',
       '2/8/2011-MAR not sent from NH for last d_med-days_since_last_value',
       '25HVD3_root-apex_value', '25HVD3_root-baseline_value',
       '25HVD3_root-first_value', '25HVD3_root-last_value',
       '25HVD3_root-nadir_value', '25HVDT_root-apex_value',
       '25HVDT_root-baseline_value', '25HVDT_root-first_value',
       '25HVDT_root-last_value', '25HVDT_root-nadir_value'], dtype='<U73')

In [263]:
features.loc[: ,union_selected_features].isnull().sum().describe()

count    3476.000000
mean      118.714902
std        74.855172
min         0.000000
25%        18.000000
50%       167.000000
75%       177.000000
max       177.000000
dtype: float64

In [257]:
np.savetxt(out_file, union_selected_features, fmt='%s', delimiter=',')